## Advanced Code for RNN

### Notes

* Does not support COCOB optimizer.
* Does not have peephole options for cells.
* Recursive approach to forecasts - always produces forecast of horizon 1 and uses as input for next period.
* Using L1 loss.
* Hyperparameters in DARTS package available to tune (intervals are from Hewamalage, Bergmeir, Bandara (2021):
    * ~~Batch size: 40-100~~
    * ~~Epochs: 3-30~~
    * ~~Dropout~~
    * ~~L2 Reg: 0.0001-0.0008~~
    * ~~Cell dimension: 20-50~~
    * ~~Number layers: 1-2~~
    * Standard deviation of initializer: 0.0001-0.0008
    * ~~learning rate (optimizer kwargs): 0.001-0.1~~
* Put in de-trending step

### Importing Modules and Data

In [1]:
from darts import TimeSeries
import pandas as pd
import numpy as np

from data_processing_functions import *
from data_protection_functions import *

from darts.models.forecasting.rnn_model import RNNModel
from sktime.performance_metrics.forecasting import mean_absolute_error

from torch import nn

from bayes_opt import BayesianOptimization

from sktime.utils.plotting import plot_series

In [78]:
# import monthly micro time series
# ignore header and skip the first row to use integers as column names
full_data = pd.read_csv("../../Data/Train/Clean/full_m3_monthly_micro_clean.csv", header=None, skiprows=1)

In [79]:
# convert to a list of series, potentially with different lengths
# drop NA values
full_data = [x.dropna() for _, x in full_data.iterrows()]

In [80]:
# forecast horizon
h = 1

In [81]:
# create train and test set using forecast horizon
Y = [x.iloc[:-h] for x in full_data]
Test = [x.iloc[-h:] for x in full_data]
Test = pd.DataFrame([x.reset_index(drop=True) for x in Test]).T

## Pre-process the Training Data

In [82]:
# deseasonalize the data
transform_dict = {"deseasonalize": {"sp": 12, "seasonality_type": "additive"}}

In [83]:
# apply differential privacy protection
Y_protected = apply_data_protection(Y, epsilon=1)

In [84]:
processed, _, _, _, _ = pre_process(ts_data=Y_protected,
                                    target_forecast_period=h,
                                    mean_normalize=True,
                                    log=True,
                                    transform_dict=transform_dict)

In [85]:
# convert to type np.float32 to speed up training
processed = [x.astype(np.float32) for x in processed]

In [86]:
# convert dataframe index to a RangeIndex
for i in range(len(processed)):
    processed[i].index = pd.RangeIndex(start=0, stop=processed[i].shape[0]) 

In [11]:
# create and remove validation set from processed
validation = [x.iloc[-h:] for x in processed]
validation = pd.DataFrame([x.reset_index(drop=True) for x in validation]).T
processed_noval = [x.iloc[:-h] for x in processed]

In [12]:
processed = [TimeSeries.from_series(x) for x in processed]
processed_noval = [TimeSeries.from_series(x) for x in processed_noval]

### Model Setup

We will use the validation set from the end of each series to optimize hyper-parameters. Then, re-train the network with the optimal parameters using the full training series, then evaluate on the test data.

***

In [13]:
def train_RNN(train_data,
              save_models,
              model_save_folder,
              h,
              num_ensemble_models,
              input_chunk_length,
              training_length,
              max_samples_per_ts,
              learning_rate_,
              n_rnn_layers_,
              hidden_dim_,
              batch_size_,
              n_epochs_,
              dropout_,
              L2_penalty_):
    
    num_series = len(train_data)
    
    full_forecasts = np.zeros([h, num_series, num_ensemble_models])
    
    for m in range(num_ensemble_models):
        
        # optimizer kwargs
        optimizer_kwargs = {'lr': learning_rate_,
                            'weight_decay': L2_penalty_}
        
        # instantiate the model
        RNN = RNNModel(input_chunk_length=input_chunk_length,
                       training_length=training_length, 
                       model="LSTM",
                       loss_fn=nn.L1Loss(),
                       optimizer_kwargs=optimizer_kwargs,
                       n_rnn_layers=n_rnn_layers_,
                       hidden_dim=hidden_dim_,
                       batch_size=batch_size_,
                       n_epochs=n_epochs_,
                       dropout=dropout_,
                       force_reset=True)
    
        # fit the model
        RNN.fit(series=train_data, max_samples_per_ts=max_samples_per_ts)
        
        if save_models:
            RNN.save_model("../../Outputs/RNN_models/" + model_save_folder + "/rnn_mod_" + str(m) + "_.pth.tar")
    
        # generate forecasts
        fcasts = RNN.predict(n=h, series=train_data)
        
        # convert to series
        fcasts = [x.pd_series().reset_index(drop=True) for x in fcasts]
        
        # convert to dataframe
        # f = f.pd_dataframe()
        
        fcasts = pd.DataFrame(fcasts).T

        # store forecasts in appropriate indexes
        full_forecasts[:,:,m] = fcasts
        
    # take the median of the forecasts
    median_forecasts = pd.DataFrame(np.median(full_forecasts, axis=2))
        
    return median_forecasts

In [14]:
def optimize_RNN(train_data,
                 save_models,
                 model_save_folder,
                 validation_data,
                 h,
                 num_ensemble_models,
                 input_chunk_length,
                 training_length,
                 max_samples_per_ts):
    
    def evaluate_RNN(learning_rate_, n_rnn_layers_, hidden_dim_, batch_size_, n_epochs_, dropout_, L2_penalty_):
        
        fcasts = train_RNN(train_data=train_data,
                           save_models=save_models,
                           model_save_folder=model_save_folder,
                           h=h,
                           num_ensemble_models=num_ensemble_models,
                           input_chunk_length=input_chunk_length,
                           training_length=training_length,
                           max_samples_per_ts=max_samples_per_ts,
                           n_rnn_layers_=int(n_rnn_layers_),
                           learning_rate_=learning_rate_,
                           hidden_dim_=int(hidden_dim_),
                           batch_size_=int(batch_size_),
                           n_epochs_=int(n_epochs_),
                           dropout_=dropout_,
                           L2_penalty_=L2_penalty_)
        
        # compute MAE
        return -mean_absolute_error(validation_data, fcasts)
         
    
    optimizer = BayesianOptimization(
        f=evaluate_RNN,
        pbounds={
            "learning_rate_": (0.001, 0.1),
            "n_rnn_layers_": (1, 2.99),
            "hidden_dim_": (20, 50.99),
            "batch_size_": (40, 100.99),
            "n_epochs_": (3, 30.99),
            "dropout_": (0.1, 0.5),
            "L2_penalty_": (0.0001, 0.0008)
        },
        random_state=1234,
        verbose=2)
    
    optimizer.maximize()
    
    print("Final Result: ", optimizer.max)
    
    return optimizer.max

In [15]:
input_chunk_length = 24
training_length = 30
max_samples_per_ts = 5
num_ensemble_models = 10

In [16]:
best_params = optimize_RNN(train_data=processed_noval,
                           save_models=False,
                           model_save_folder=None,
                           validation_data=validation,
                           h=h,
                           num_ensemble_models=num_ensemble_models,
                           input_chunk_length=input_chunk_length,
                           training_length=training_length,
                           max_samples_per_ts=max_samples_per_ts)

[2022-07-01 10:42:20,208] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
[2022-07-01 10:42:20,208] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
2022-07-01 10:42:20 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2370 samples.
C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.27509109560284584 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2022-07-01 10:42:20,

|   iter    |  target   | L2_pen... | batch_... | dropout_  | hidden... | learni... | n_epochs_ | n_rnn_... |
-------------------------------------------------------------------------------------------------------------


Training: 0it [00:00, ?it/s]

Predicting: 31it [00:00, ?it/s]

[2022-07-01 10:42:33,446] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
[2022-07-01 10:42:33,446] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
2022-07-01 10:42:33 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2370 samples.
C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.27509109560284584 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2022-07-01 10:42:33,

...

Predicting: 36it [00:00, ?it/s]

[2022-07-01 13:20:03,363] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
[2022-07-01 13:20:03,363] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
2022-07-01 13:20:03 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2370 samples.
C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2022-07-01 13:20:03,368] INFO | dart

Training: 0it [00:00, ?it/s]

Predicting: 36it [00:00, ?it/s]

|  30       | -2.44     |  0.0008   |  66.35    |  0.1      |  42.78    |  0.1      |  23.04    |  1.0      |
Final Result:  {'target': -2.439235970543212, 'params': {'L2_penalty_': 0.0008, 'batch_size_': 69.47039148912809, 'dropout_': 0.28913695641193615, 'hidden_dim_': 47.12083346156072, 'learning_rate_': 0.1, 'n_epochs_': 19.454285395692697, 'n_rnn_layers_': 1.0}}


Now, train RNN on full training data using the optimal parameters.

In [86]:
fcasts = train_RNN(train_data=processed,
                   save_models=True,
                   model_save_folder="DP_1",
                   h=h,
                   num_ensemble_models=num_ensemble_models,
                   input_chunk_length=input_chunk_length,
                   training_length=training_length,
                   max_samples_per_ts=max_samples_per_ts,
                   learning_rate_=best_params['params']['learning_rate_'],
                   n_rnn_layers_=int(best_params['params']['n_rnn_layers_']),
                   hidden_dim_=int(best_params['params']['hidden_dim_']),
                   batch_size_=int(best_params['params']['batch_size_']),
                   n_epochs_=int(best_params['params']['n_epochs_']),
                   dropout_=best_params['params']['dropout_'],
                   L2_penalty_=best_params['params']['L2_penalty_'])

[2022-07-01 16:08:22,286] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
[2022-07-01 16:08:22,286] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
2022-07-01 16:08:22 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2370 samples.
C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28913695641193615 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2022-07-01 16:08:22,

Training: 0it [00:00, ?it/s]

Predicting: 35it [00:00, ?it/s]

[2022-07-01 16:08:49,681] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
[2022-07-01 16:08:49,681] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
2022-07-01 16:08:49 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2370 samples.
C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28913695641193615 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2022-07-01 16:08:49,

Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Predicting: 35it [00:00, ?it/s]

[2022-07-01 16:13:18,080] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
[2022-07-01 16:13:18,080] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2370 samples.
2022-07-01 16:13:18 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2370 samples.
C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28913695641193615 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2022-07-01 16:13:18,

Training: 0it [00:00, ?it/s]

Predicting: 35it [00:00, ?it/s]

***

***

In [ ]:
# model_loaded = RNNModel.load_model("../../Outputs/RNN_models/rnn_mod_1_.pth.tar")

In [21]:
# generate forecasts
# fcasts = model_loaded.predict(n=h, series=processed)
        
# convert to series
# fcasts = [x.pd_series().reset_index(drop=True) for x in fcasts]
        
# fcasts = pd.DataFrame(fcasts).T

In [87]:
fcasts = [pd.Series(fcasts.iloc[:,i]) for i in range(fcasts.shape[1])]

In [88]:
fcasts = post_process(full_ts_data=Y_protected,
                      forecasts=fcasts,
                      target_forecast_period=h,
                      mean_normalize=True,
                      log=True,
                      transform_dict=transform_dict)

In [89]:
# Test_cluster = [Test[i] for i in length_clusters[cluster_num]]

In [90]:
# Test_cluster = pd.DataFrame([x.reset_index(drop=True) for x in Test_cluster]).T

In [91]:
mean_absolute_error(Test, fcasts)

1475.7874625329835

In [92]:
best_params

{'target': -2.439235970543212,
 'params': {'L2_penalty_': 0.0008,
  'batch_size_': 69.47039148912809,
  'dropout_': 0.28913695641193615,
  'hidden_dim_': 47.12083346156072,
  'learning_rate_': 0.1,
  'n_epochs_': 19.454285395692697,
  'n_rnn_layers_': 1.0}}

***

***

### Loading RNN Models and Generating Fitted Values

In [16]:
full_fitted_vals = []

for i in range(10):
    
    # load model
    model_loaded = RNNModel.load_model("../../Outputs/RNN_models/DP_1/rnn_mod_" + str(i) + "_.pth.tar")
    
    # generate fitted values
    fitted = [model_loaded.historical_forecasts(series=x, start=24, retrain=False) for x in processed[:5]]
    
    # convert to series
    fitted = [x.pd_series().reset_index(drop=True) for x in fitted]
          
    # convert to dataframe
    fitted = pd.DataFrame(fitted).T
    
    # append fitted values
    full_fitted_vals.append(fitted)

C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28913695641193615 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
2022-07-09 13:50:35 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-07-09 13:50:35 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-09 13:50:35 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-09 13:50:35 pytorch_lightning.utilities.rank_zero INFO: HPU avai

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [17]:
full_fitted_vals = np.dstack(full_fitted_vals)

In [28]:
series_three_fitted = full_fitted_vals[:,2,:]

In [25]:
med_fitted = np.median(full_fitted_vals, axis=2)

In [29]:
med_fitted = pd.DataFrame(med_fitted)

In [30]:
series_three_fitted = pd.DataFrame(series_three_fitted)

In [31]:
med_fitted = [pd.Series(med_fitted.iloc[:,i]) for i in range(med_fitted.shape[1])]

In [32]:
series_three_fitted = [pd.Series(series_three_fitted.iloc[:,i]) for i in range(series_three_fitted.shape[1])]

In [33]:
for s in med_fitted:
    s.index = np.arange(input_chunk_length, input_chunk_length+len(s))

In [34]:
for s in series_three_fitted:
    s.index = np.arange(input_chunk_length, input_chunk_length+len(s))

In [35]:
med_fitted = post_process(full_ts_data=Y_protected[:5],
                          forecasts=med_fitted,
                          target_forecast_period=h,
                          mean_normalize=True,
                          log=True,
                          transform_dict=transform_dict)

In [37]:
protected_series_three = [Y_protected[2] for i in range(10)]

In [39]:
series_three_fitted = post_process(full_ts_data=protected_series_three,
                          forecasts=series_three_fitted,
                          target_forecast_period=h,
                          mean_normalize=True,
                          log=True,
                          transform_dict=transform_dict)

In [178]:
med_fitted = med_fitted.apply(np.log)

In [179]:
med_fitted.to_csv("../../Outputs/Tables/rnn_dp_1_fitted_vals.csv", index=False)

In [43]:
series_three_fitted = series_three_fitted.apply(np.log)

In [44]:
series_three_fitted.to_csv("../../Outputs/Tables/rnn_dp_1_series_3_fitted_vals.csv", index=False)

## MAE for Each RNN

In [59]:
num_series = len(processed)

full_forecasts = np.zeros([h, num_series, num_ensemble_models])

for i in range(num_ensemble_models):
    
    # load model
    model_loaded = RNNModel.load_model("../../Outputs/RNN_models/DP_1/rnn_mod_" + str(i) + "_.pth.tar")

    # generate forecasts
    fcasts = model_loaded.predict(n=h, series=processed)
        
    # convert to series
    fcasts = [x.pd_series().reset_index(drop=True) for x in fcasts]
        
    fcasts = pd.DataFrame(fcasts).T

    # store forecasts in appropriate indexes
    full_forecasts[:,:,i] = fcasts

C:\Users\Cameron\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
C:\Users\Cameron\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28913695641193615 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
2022-07-09 17:17:59 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-07-09 17:17:59 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-09 17:17:59 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-09 17:17:59 pytorch_lightning.utilities.rank_zero INFO: HPU avai

Predicting: 0it [00:00, ?it/s]

2022-07-09 17:18:01 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-07-09 17:18:01 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-09 17:18:01 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-09 17:18:01 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

2022-07-09 17:18:02 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-07-09 17:18:02 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-09 17:18:02 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-09 17:18:02 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs


...

Predicting: 0it [00:00, ?it/s]

2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-09 17:18:08 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [64]:
fcast_list = [pd.DataFrame(full_forecasts[:,:,i]) for i in range(num_ensemble_models)]

In [68]:
fcast_list = [[pd.Series(fcast_list[x].iloc[:,i]) for i in range(fcast_list[x].shape[1])] for x in range(num_ensemble_models)]

In [73]:
fcast_indexes = [np.arange(Y[i].index[-1]+1, Y[i].index[-1]+h+1) for i in range(num_series)]

# add correct time index back to forecasts
for i in range(num_ensemble_models):
    for j in range(num_series):
        fcast_list[i][j].index = fcast_indexes[j]

In [89]:
post_list = [post_process(full_ts_data=Y_protected,
                          forecasts=x,
                          target_forecast_period=h,
                          mean_normalize=True,
                          log=True,
                          transform_dict=transform_dict) for x in fcast_list]

In [92]:
maes = [mean_absolute_error(Test, x) for x in post_list]

In [93]:
maes

[1451.8241512439809,
 1769.6849659246327,
 1388.5800947676032,
 1602.3334127561773,
 1392.3447195027918,
 1386.6022264495946,
 1319.2288331369164,
 1328.200552454837,
 1426.4998597613517,
 1451.9693264311802]